In [ ]:
pip install transformers torch pillow matplotlib

In [ ]:
pip install timm

In [1]:
from PIL import Image, ImageDraw, ImageFont
import torch
from transformers import AutoImageProcessor, AutoModelForObjectDetection

ckpt = "yainage90/fashion-object-detection"

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load
processor = AutoImageProcessor.from_pretrained(ckpt)
model = AutoModelForObjectDetection.from_pretrained(ckpt).to(device).eval()

# image 
img_path = r"C:\Users\lamia\OneDrive\الصور\لقطات الشاشة\Screenshot 2026-01-04 001635.png"
image = Image.open(img_path).convert("RGB")
image.show()
# inference
inputs = processor(images=image, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)

# post-process
target_sizes = torch.tensor([[image.size[1], image.size[0]]], device=device)
results = processor.post_process_object_detection(
    outputs, threshold=0.4, target_sizes=target_sizes
)[0]

# draw
draw = ImageDraw.Draw(image)
try:
    font = ImageFont.truetype("arial.ttf", 16)
except:
    font = ImageFont.load_default()

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    score = float(score)
    label = int(label)
    xmin, ymin, xmax, ymax = [int(x) for x in box.tolist()]

    name = model.config.id2label[label]
    text = f"{name} ({score:.2f})"

    draw.rectangle([(xmin, ymin), (xmax, ymax)], outline="red", width=3)
    draw.text((xmin, max(0, ymin-18)), text, fill="red", font=font)

# save
out_path = "detections.png"
image.save(out_path)
print("Saved:", out_path)


C:\Users\lamia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
C:\Users\lamia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\

Saved: detections.png
